In [10]:
import requests as req
import os
import json
from pprint import pprint

import numpy as np
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000


In [13]:
#### Download Google Spreadsheets ####

# Make a temporary directory, and move into it
!mkdir temp
os.chdir("temp")
dest = os.getcwd()

# July Data Sheet
!curl "https://docs.google.com/spreadsheets/d/1viPOGYIk6RGu7YMoM3BHNVbkWaCZ0JFBOMSNncWvHYk/export?format=tsv" > july_data_upload.tsv
july_data_upload = pd.read_csv(dest+"/july_data_upload.tsv", sep="\t", index_col=[0])

# # Metadata to Upload
# !curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
# current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
# current_mdata = current_mdata.transpose()

!curl "https://docs.google.com/spreadsheets/d/1M_hejv-nSUXSldocNwEQNQ7SxEJkN_exk6krcXnq6VI/export?format=tsv" > current_metadata.tsv
current_mdata = pd.read_csv(dest+"/current_metadata.tsv", sep="\t", index_col=[0])
current_mdata = current_mdata.transpose()

# Delete temporary files
os.chdir("..")
!rm -r temp


#### Merge info from the Tracking sheet and Metadata sheet

# Add RW API key to metadata_to_upload
def fetch_info(row, match_col, dest_df, target_column):
    try:
        info = dest_df.loc[row[match_col], target_column]
        return(info)
    except:
        return(None)

old_id_col = "VIZZ Unique ID - RW API (bulk upload)"
new_id_col = "API-ID (PERFECT DATASET)"
dl_from_src_col = "Download from Source"
dl_from_s3_col = "Download Data (S3)"

match_col = "Unique ID"

current_mdata[old_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,old_id_col), axis=1)
current_mdata[new_id_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,new_id_col), axis=1)

current_mdata[dl_from_src_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_src_col), axis=1)
current_mdata[dl_from_s3_col] = current_mdata.apply(lambda row: fetch_info(row,match_col,july_data_upload,dl_from_s3_col), axis=1)

# Keep only those datasets with rw_ids already
valid_old_ids = pd.notnull(current_mdata[old_id_col])
valid_new_ids = pd.notnull(current_mdata[new_id_col])

def choose_new_id(df, valid_old_ids, valid_new_ids):
    assert(len(valid_old_ids)==len(valid_new_ids))
    final_ids = []
    for i in range(0, len(valid_new_ids)):
        if(valid_new_ids[i]):
            final_ids.append(df.iloc[i][new_id_col])
        elif(valid_old_ids[i]):
            final_ids.append(df.iloc[i][old_id_col])
        else:
            final_ids.append(None)
    return(final_ids)

current_mdata["final_ids"] = choose_new_id(current_mdata, valid_old_ids, valid_new_ids)
keep_matched_ids = pd.notnull(current_mdata["final_ids"])

current_mdata = current_mdata.loc[keep_matched_ids]
current_mdata.set_index("final_ids", inplace=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92220    0 92220    0     0   134k      0 --:--:-- --:--:-- --:--:--  134k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  577k    0  577k    0     0   421k      0 --:--:--  0:00:01 --:--:--  421k


In [14]:
print(current_mdata.shape)
print(july_data_upload.shape)

(91, 33)
(290, 35)


In [15]:
current_mdata.head(1)

,Public Title,Technical Title,Subtitle,Unique ID,Source Organizations,Learn More Link,Function,Description,Cautions,Geographic Coverage,Data Type,Spatial Resolution,Date of Content,Frequency of Updates,Summary of Licence,Link to License,Citation,Published Language,Published Title (if not English),Download,Download from Source,Uploaded To,Layer Name 1,Layer Definition 1,Layer Name 2,Layer Definition 2,Layer Name 3,Layer Definition 3,Layer Name 4,Layer Definition 4,VIZZ Unique ID - RW API (bulk upload),API-ID (PERFECT DATASET),Download Data (S3)
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93ee443e-cb39-424a-9aa4-1d16af813418,Endangered Species Sites,NaN,AZE,bio.001,Alliance for Zero Extinction (AZE),http://www.biodiversitya-z.org/content/allianc...,Global list of sites containing 95% or more of...,Created by the Alliance for Zero Extinction (A...,None listed.,Global,Vector,NaN,2010,5 years,The AZE data and any derivatives may be used f...,https://www.arcgis.com/home/item.html?id=4ecca...,"Alliance for Zero Extinction. 2010. ""2010 AZE ...",English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93ee443e-cb39-424a-9aa4-1d16af813418,93ee443e-cb39-424a-9aa4-1d16af813418,NaN


In [16]:
july_data_upload.head(1)

,VIZZ Unique ID - RW API (bulk upload),Slug - RW API (sanity check),API-ID (PERFECT DATASET),Public Title,Alternative Public Title,Technical Title,Subtitle,Theme_1,Theme_2,Theme_3,Planet Pulse,Water & Conflict,Problem Solving,Process these first,Multiple Layers needed to surface at same time?,Metadata Completed,Metadata on Backoffice,Distribution Restriction,Tags,Format,"Endpoint URL (Carto, GEE)",Download from Source,Download Data (S3),On WRI Platform,Dataset Processed for Upload,Data Upload Responsibility,Uploaded to S3,Server Location,Missing ISO Code,Dataset on Backoffice,Columns defined on Backoffice,Layer Definition,Layer Name/Description finalized,Widgets,Published on RW
WRI Unique ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
com.011.1,/bf5877eb-399a-4237-b510-b1d41049e3bc,NaN,NaN,Maritime Boundaries - Territorial Waters (12nm),NaN,Maritime Boundaries - Territorial Waters (12nm),Marine Regions,Commerce,NaN,NaN,NaN,NaN,Peter to check - do separate datasets as needed,NaN,NaN,X,NaN,X (Attribution required. Redistribution not pe...,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://www.marineregions.org/downloads.php,NaN,NaN,NaN,Peter,X,Carto,NaN,X,NaN,X,NaN,NaN,X


In [17]:
current_mdata.columns

Index(['Public Title', 'Technical Title', 'Subtitle', 'Unique ID',
       'Source Organizations', 'Learn More Link', 'Function', 'Description',
       'Cautions', 'Geographic Coverage', 'Data Type', 'Spatial Resolution',
       'Date of Content', 'Frequency of Updates', 'Summary of Licence',
       'Link to License', 'Citation', 'Published Language',
       'Published Title (if not English)', 'Download', 'Download from Source',
       'Uploaded To', 'Layer Name 1', 'Layer Definition 1', 'Layer Name 2',
       'Layer Definition 2', 'Layer Name 3', 'Layer Definition 3',
       'Layer Name 4', 'Layer Definition 4',
       'VIZZ Unique ID - RW API (bulk upload)', 'API-ID (PERFECT DATASET)',
       'Download Data (S3)'],
      dtype='object')

In [19]:
## FOR EACH DATASET IN BACKOFFICE THAT HAS METADATA, UPLOAD IT
from configparser import ConfigParser
config = ConfigParser()
config.read("../.env")
api_token = config.get("configuration", "api_token")

auth_token = api_token # <Insert Auth Token Here>

for rw_id in current_mdata.index:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    print(url)
    #url= 'https://api.resourcewatch.org/v1/dataset/9e9a5c50-b825-4f12-838f-1650943c2be1/metadata'
    # Everything from current_mdata
    metadata = current_mdata.loc[rw_id]
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": metadata["Public Title"],
        "description": metadata["Description"],
        "source": metadata["Subtitle"],
        "functions": metadata["Function"],
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            "wri_rw_id": metadata["Unique ID"],
            "data_type": metadata["Data Type"],

            "name": metadata["Public Title"],
            "source_organization": metadata["Source Organizations"],
            "technical_title":metadata["Technical Title"],
            
            "function": metadata["Function"],
            "cautions": metadata["Cautions"],
            
            "citation": metadata["Citation"],
            "summary_of_license": metadata["Summary of Licence"],
            "link_to_license": metadata["Link to License"],
            
            "geographic_coverage": metadata["Geographic Coverage"],
            "spatial_resolution": metadata["Spatial Resolution"],
            "date_of_content": metadata["Date of Content"],
            "frequency_of_updates": metadata["Frequency of Updates"],
            
            "learn_more_link": metadata["Learn More Link"],
            "data_download_link": metadata["Download Data (S3)"],
            "data_download_original_link": metadata["Download from Source"]
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }
    #print(row_payload)

    try:
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])

https://api.resourcewatch.org/v1/dataset/93ee443e-cb39-424a-9aa4-1d16af813418/metadata
https://api.resourcewatch.org/v1/dataset/4458eb12-8572-45d1-bf07-d5a3ee097021/metadata
https://api.resourcewatch.org/v1/dataset/16df8ada-87cc-4907-adce-a98bc4e91856/metadata
https://api.resourcewatch.org/v1/dataset/3624554e-b240-4edb-9110-1f010642c3f3/metadata
https://api.resourcewatch.org/v1/dataset/ad790c87-fe9e-4405-891d-de7c2ddfda79/metadata
https://api.resourcewatch.org/v1/dataset/7d3465f8-5959-4531-aaf2-c9a8a03183b3/metadata
https://api.resourcewatch.org/v1/dataset/de452a4c-a55c-464d-9037-8c3e9fe48365/metadata
https://api.resourcewatch.org/v1/dataset/3c12072d-611b-413f-b314-4df0834523ab/metadata
https://api.resourcewatch.org/v1/dataset/33bed1fb-9261-41bf-8b50-127a4d0c80c5/metadata
https://api.resourcewatch.org/v1/dataset/a07f1bed-ca16-4fbf-b14b-d3a0344cab74/metadata
https://api.resourcewatch.org/v1/dataset/36803484-c413-49a9-abe2-2286ee99b624/metadata
https://api.resourcewatch.org/v1/dataset/05

In [205]:
#### Merge subtitles with Tracking sheet


#### Find out which datasets have a Unique ID, but aren't 
# Keep only those datasets with rw_ids already
tracking_valid_old_ids = pd.notnull(july_data_upload[old_id_col])
tracking_valid_new_ids = pd.notnull(july_data_upload[new_id_col])

july_data_upload["final_ids"] = choose_new_id(july_data_upload, tracking_valid_old_ids, tracking_valid_new_ids)

missed_ids = [rw_id for rw_id in july_data_upload["final_ids"].values if ((rw_id not in current_mdata.index) and (rw_id != None))]

missed_data = july_data_upload.reset_index().set_index("final_ids")
missed_data = missed_data.loc[missed_ids]
missed_data

for rw_id in missed_data.index:
    url = "https://api.resourcewatch.org/v1/dataset/"+str(rw_id)+"/metadata"
    print(url)
    # Everything from current_mdata
    metadata = missed_data.loc[rw_id]
    print(metadata["WRI Unique ID"])
    print(metadata["Public Title"])
    #print(metadata)
    row_payload = {
        "language": "en",
        
        "name": clean_nulls(metadata["Public Title"]),
        "source": clean_nulls(metadata["Subtitle"]),
        
        "application":"rw",
        "dataset":rw_id,
        
        "info": {
            
            "wri_rw_id": clean_nulls(metadata["WRI Unique ID"]),

            "name": clean_nulls(metadata["Public Title"]),
            "technical_title":clean_nulls(metadata["Technical Title"]),

            "data_download_link": [clean_nulls(metadata["Download Data (S3)"]), clean_nulls(metadata["Download from Source"])],
            "download_data_s3": clean_nulls(metadata["Download Data (S3)"]),
            "download_from_source": clean_nulls(metadata["Download from Source"]),
            
        }
    }

    headers = {
        'content-type': "application/json",
        'authorization': "Bearer " + auth_token,
    }
    #print(row_payload)

    try:
        res = req.request("POST", url, data=json.dumps(row_payload), headers = headers)
        if("already exists" in res.text):
            res = req.request("PATCH", url, data=json.dumps(row_payload), headers = headers)
            if("errors:" in res.text):
                print(res.text)
        elif("errors:" in res.text):
            print(res.text)
    except TypeError as e:
        print(e.args)
        print(metadata[["Unique ID", "Public Title"]])



https://api.resourcewatch.org/v1/dataset/894f43a8-ce8e-43a5-a4c7-fa80faa43d63/metadata
bio.031
Social and Economic Dependence on Coral Reefs
https://api.resourcewatch.org/v1/dataset/6b8442f5-4766-4444-94b4-d6676277fd80/metadata
bio.033
Cold Water Corals
https://api.resourcewatch.org/v1/dataset/7d9c0d09-e833-4a74-811b-0af78da9c731/metadata
cit.032
Urban Population Projections
https://api.resourcewatch.org/v1/dataset/c8040a7a-a40f-48bd-b003-625c33beff5e/metadata
cli.003
Sea Surface Temperature Anomalies
https://api.resourcewatch.org/v1/dataset/c9c9cb2f-9655-4f40-8736-9b407ee43514/metadata
cli.013
CH4 Concentrations
https://api.resourcewatch.org/v1/dataset/ea67f436-473c-4977-bdf2-8aa0dabbaa6f/metadata
cli.021
Average Snow Cover
https://api.resourcewatch.org/v1/dataset/d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a/metadata
cli.025
Relative Sea Level Trends
https://api.resourcewatch.org/v1/dataset/a5e50799-fbd1-4aca-b3fa-22a6f6fa7aad/metadata
dis.002.1
Hurricane Tracks
https://api.resourcewatch.org/

In [204]:
missed_data

,WRI Unique ID,Unique ID - RW API (used for bulk upload),Slug - RW API (sanity check),API-ID (PERFECT DATASET),Public Title,Alternative Public Title,Technical Title,Subtitle,Theme_1,Theme_2,Theme_3,Planet Pulse,Water & Conflict,Problem Solving,Process these first,Multiple Layers needed to surface at same time?,Metadata Completed,Metadata on Backoffice,Distribution Restriction,Tags,Format,"Endpoint URL (Carto, GEE)",Download from Source,Download Data (S3),On WRI Platform,Dataset Processed for Upload,Data Upload Responsibility,Uploaded to S3,Server Location,Missing ISO Code,Dataset on Backoffice,Columns defined on Backoffice,Layer Definition,Layer Name/Description finalized,Widgets,Published on RW
final_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
894f43a8-ce8e-43a5-a4c7-fa80faa43d63,bio.031,NaN,NaN,894f43a8-ce8e-43a5-a4c7-fa80faa43d63,Social and Economic Dependence on Coral Reefs,NaN,Social and Economic Dependence on Coral Reefs,WRI,Biodiversity,Commerce,Society,NaN,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://datasets.wri.org/dataset/reefs-at-risk-...,NaN,NaN,NaN,Liz,NaN,Carto,NaN,X,NaN,X,X,NaN,X
6b8442f5-4766-4444-94b4-d6676277fd80,bio.033,NaN,NaN,6b8442f5-4766-4444-94b4-d6676277fd80,Cold Water Corals,NaN,Global DIstribution of Cold Water Corals,UNEP-WCMC,Biodiversity,NaN,NaN,NaN,NaN,Max/Laura write metadata,NaN,NaN,ready,NaN,X,NaN,Vector,https://resourcewatch.carto.com/u/wri-rw/datas...,http://data.unep-wcmc.org/datasets/3,https://wri-public-data.s3.amazonaws.com/resou...,NaN,NaN,Elise,X,Carto,NaN,X,NaN,X,X,NaN,X
7d9c0d09-e833-4a74-811b-0af78da9c731,cit.032,7d9c0d09-e833-4a74-811b-0af78da9c731,NaN,NaN,Urban Population Projections,NaN,Urban Population Percentage,No Metadata Yet,Cities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ready,NaN,X,NaN,CSV,https://resourcewatch.carto.com/u/wri-rw/datas...,https://esa.un.org/unpd/wup/CD-ROM/,https://wri-projects.s3.amazonaws.com/resource...,NaN,NaN,Emily,X,Carto,NaN,X,NaN,X,NaN,X,X
c8040a7a-a40f-48bd-b003-625c33beff5e,cli.003,c8040a7a-a40f-48bd-b003-625c33beff5e,Sea-Surface-Temperature-Anomalies-NOAA,NaN,Sea Surface Temperature Anomalies,NaN,Twice-weekly Sea Surface Temperature Anomalies,No Metadata Yet,Climate,NaN,NaN,RT,NaN,Vizz rasters - who makes layer?,NaN,NaN,ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,?,NaN,NaN,X
c9c9cb2f-9655-4f40-8736-9b407ee43514,cli.013,c9c9cb2f-9655-4f40-8736-9b407ee43514,Current-Methane-Concentration-CH4,NaN,CH4 Concentrations,NaN,"Current Methane Concentration, CH4",No Metadata Yet,Climate,NaN,NaN,RT,NaN,Liz to check,NaN,NaN,Liz,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,X,NaN,NaN,NaN
ea67f436-473c-4977-bdf2-8aa0dabbaa6f,cli.021,ea67f436-473c-4977-bdf2-8aa0dabbaa6f,NaN,NaN,Average Snow Cover,NaN,Northern Hemisphere Average Snow Cover by Mont...,No Metadata Yet,Climate,NaN,NaN,RT,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,Raster,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN
d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,cli.025,d1fd9ac1-b99f-4f5f-b9d8-d3f1f818321a,Sea-Level-Trends,NaN,Relative Sea Level Trends,NaN,Global Linear Relative Mean Sea Level (MSL) tr...,No Metadata Yet,Climate,Cities,NaN,explore,NaN,NaN,NaN,NaN,ready,NaN,NaN,NaN,NaN,NaN,https://tidesandcurrents.noaa.gov/sltrends/msl...,NaN,NaN,NaN,Vizz,NaN,Carto,NaN,X,NaN,X,NaN,NaN,NaN
a5e50799-fbd1-4aca-b3fa-22a6f6fa7aad,dis.002.1,a5e50799-fbd1-4aca-b3fa-22a6f6fa7aad,Hurricane-Tracks-Cat-1-2-Hurricanes-Small-Scal...,NaN,Hurricane Tracks,NaN,"Hurricane Tracks: Category 1-2 Hurricanes, Sma...",No Metadata Yet,Disasters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Vector,NaN,NaN,NaN,NaN,NaN,Vizz,NaN,featureservice,NaN,X,NaN,X,NaN,NaN,X
fa83e8c5-e83f-4488-bd93-7c8d0ba205c1,dis.002.2,fa83e8c5-e83f-4488-bd93-7c8d0ba205c1,Hurricane-Tracks-Cat-1-2-Hurricanes-Large-Scal...,NaN,Hurricane Tracks,NaN,"Hurricane Tracks: Category 1-2 Hurricanes, Lar...",No Metadata Yet,Disasters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Vector,NaN,NaN,NaN,NaN,NaN,Viz

In [64]:
#### Inspect metadata on backoffice



# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.set_index("rw_id", inplace=True)
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

In [66]:
current_datasets_on_api.loc["0087944f-871c-44bc-b4d9-cd5acfc27023", "layers"]

[{'attributes': {'application': ['rw'],
   'applicationConfig': {},
   'dataset': '0087944f-871c-44bc-b4d9-cd5acfc27023',
   'default': True,
   'description': '',
   'env': 'production',
   'interactionConfig': {},
   'iso': [],
   'layerConfig': {'body': {'attribution': '2016 wri/google/Hansen',
     'errorTileUrl': 'https://storage.googleapis.com/global-surface-water/downloads_ancillary/blank.png',
     'format': 'image/png',
     'maxZoom': 13,
     'transparent': True},
    'service': 'leaflet',
    'type': 'tileLayer',
    'url': 'http://staging-api.globalforestwatch.org/v1/true-color-tiles/loss/{z}/{x}/{y}'},
   'legendConfig': {'items': [{'color': '#dc1e8c',
      'name': 'Loss (2001-2016)'}],
    'type': 'basic'},
   'name': 'Forest loss',
   'provider': 'leaflet',
   'slug': 'Forest-loss',
   'staticImageConfig': {},
   'updatedAt': '2017-10-06T10:07:30.025Z',
   'userId': '57a0aa1071e394dd32ffe137'},
  'id': '6d29ffbc-11d3-4cd7-b2d5-25f0ff7ffb77',
  'type': 'layer'},
 {'attr